In [11]:
import pandas as pd
data = pd.read_csv("../data/bad_speaker_turns_example.csv")
data["turnText"] = data["turnText"].fillna("").astype(str)
data.head()

,mfcc1_sma3Mean,mfcc2_sma3Mean,mfcc3_sma3Mean,mfcc4_sma3Mean,F0semitoneFrom27_5Hz_sma3nzMean,F1frequency_sma3nzMean,turnText,speaker,startTime,endTime,duration,mp3url,turnCount,inferredSpeakerRole,inferredSpeakerName,predictedClassIndex
0,29.443323,2.191056,-0.515787,-1.080479,15.734092,683.053101,[Music] [Music] The idea,SPEAKER_02,0.00,41.00,41.00,https://chtbl.com/track/G42DE4/https://dts.pod...,1,NO_INFERRED_ROLE,NO_INFERRED_SPEAKER,NaN
1,22.094163,2.524928,-0.993555,-9.593185,3.092211,726.254353,for,"SPEAKER_02, SPEAKER_01",41.00,41.30,0.30,https://chtbl.com/track/G42DE4/https://dts.pod...,2,NO_INFERRED_ROLE,NO_INFERRED_SPEAKER,NaN
2,12.802304,5.129226,-6.618303,-11.544758,0.783188,648.186766,all,SPEAKER_01,41.30,41.84,0.54,https://chtbl.com/track/G42DE4/https://dts.pod...,3,NO_INFERRED_ROLE,NO_INFERRED_SPEAKER,NaN
3,29.943765,6.282519,2.623754,-1.608636,13.058075,696.948490,this,"SPEAKER_01, SPEAKER_02",41.84,42.00,0.16,https://chtbl.com/track/G42DE4/https://dts.pod...,4,NO_INFERRED_ROLE,NO_INFERRED_SPEAKER,NaN
4,20.214113,4.026327,1.215602,-2.605735,4.820311,728.451279,"really came from a dream. Yes,",SPEAKER_02,42.00,44.90,2.90,https://chtbl.com/track/G42DE4/https://dts.pod...,5,NO_INFERRED_ROLE,NO_INFERRED_SPEAKER,NaN


In [12]:
data['duration'].describe() # in seconds

count    1306.000000
mean        5.664709
std        12.798447
min         0.000000
25%         0.570000
50%         1.420000
75%         4.530000
max       136.520000
Name: duration, dtype: float64

In [14]:
# Function to sort speaker names within each cell
def sort_speakers(speaker):
    speakers = speaker.split(', ')
    speakers.sort()
    return ', '.join(speakers)

# Apply the function to the speaker column
data['speaker'] = data['speaker'].apply(sort_speakers)

# Count the occurrences of each unique sorted speaker combination
sorted_speaker_counts = data['speaker'].value_counts()
sorted_speaker_counts

speaker
SPEAKER_02                            350
SPEAKER_00                            344
SPEAKER_00, SPEAKER_02                267
SPEAKER_03                             81
SPEAKER_02, SPEAKER_03                 66
SPEAKER_00, SPEAKER_03                 49
SPEAKER_01, SPEAKER_02                 47
SPEAKER_01                             31
SPEAKER_00, SPEAKER_01                 29
SPEAKER_00, SPEAKER_02, SPEAKER_03     14
SPEAKER_00, SPEAKER_01, SPEAKER_02     14
SPEAKER_00, SPEAKER_02, SPEAKER_04      3
SPEAKER_01, SPEAKER_03                  2
SPEAKER_00, SPEAKER_01, SPEAKER_03      2
SPEAKER_02, SPEAKER_04                  2
SPEAKER_01, SPEAKER_02, SPEAKER_03      2
SPEAKER_04                              1
SPEAKER_00, SPEAKER_01, SPEAKER_04      1
SPEAKER_00, SPEAKER_04                  1
Name: count, dtype: int64

In [16]:
def reconstruct_speaker_turns(df):
    merged_turns = []
    current_turn = {"speaker": None, "startTime": None, "endTime": None, "turnText": ""}

    for idx, row in df.iterrows():
        speaker = row["speaker"]
        text = row["turnText"]
        start_time = row["startTime"]
        end_time = row["endTime"]

        # Merge if the same speaker and less than 1 second time gap
        if current_turn["speaker"] == speaker and abs(current_turn["endTime"] - start_time) < 1:
            current_turn["turnText"] += " " + text.strip()
            current_turn["endTime"] = end_time
        else:
            if current_turn["speaker"] is not None:
                merged_turns.append(current_turn)
            current_turn = {
                "speaker": speaker,
                "startTime": start_time,
                "endTime": end_time,
                "turnText": text.strip(),
            }

    # Append the last turn
    if current_turn["speaker"] is not None:
        merged_turns.append(current_turn)

    return pd.DataFrame(merged_turns)

In [17]:
reconstructed_data = reconstruct_speaker_turns(data)

In [18]:
reconstructed_data.head()

,speaker,startTime,endTime,turnText
0,SPEAKER_02,0.00,41.00,[Music] [Music] The idea
1,"SPEAKER_01, SPEAKER_02",41.00,41.30,for
2,SPEAKER_01,41.30,41.84,all
3,"SPEAKER_01, SPEAKER_02",41.84,42.00,this
4,SPEAKER_02,42.00,44.90,"really came from a dream. Yes,"
